In [1]:
import numpy as np
import pandas as pd
import pickle
import os

os.environ["CUDA_VISIBLE_DEVICES"]= '1' #set gpu 

### Load Data and Functions

In [ ]:
#Load Auxiliary functions
exec(open('FCD.py').read()) #frechet distance, keras loss functions, data_generator, CHEMBLNET

# Load generated molecules
gen_mol_file = "generated_smiles/LSTM_Segler.smi" #input file which contains one generated SMILES per line
gen_mol = pd.read_csv(gen_mol_file,header=None)[0] #IMPORTANT: take at least 10000 molecules as FCD can vary with sample size 
sample = np.random.choice(gen_mol, 10000, replace=False)
sample_std = canoncial_smiles(sample)
 #IMPORTANT: use canoncial SMILES otherwise the FCD might be not representative
# Load ChEMNet and reference stats
mu_ref, cov_ref = pickle.load(open("ref_combined_stat.p", 'rb')).values() #statistics of random 50,000 chembl molecules, which were not used for training
model = load_ref_model('ChemNet_v0.13_pretrained.h5')

Using TensorFlow backend.


## Calculation of Frechet CHEMBLNET DISTANCE

In [ ]:
#get CHEBMLNET activations of generated molecules 
gen_mol_act = get_predictions(model, sample_std)
FCD = calculate_frechet_distance(mu1=np.mean(gen_mol_act, axis=0), mu2=mu_ref, 
                                 sigma1=np.cov(gen_mol_act.T), sigma2=cov_ref)
FCD